In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

In [3]:
y = df['label']
X = df.drop(['label'], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [6]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.output_layer(out)
        return out

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [10]:
model = NeuralNetwork().to(device)

In [11]:
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)
    
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss}')
        print(f'current: {current}')

In [13]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f'test_loss: {test_loss}')
    print(f'correct: {correct}')

In [14]:
epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}")
    train_loop(train_dataloader,  model, loss, optimizer)
    test_loop(test_dataloader, model, loss)

Epoch 1
loss: 1.0461045503616333
current: 0
loss: 1.2448220252990723
current: 10
loss: 1.1061203479766846
current: 20
loss: 1.1646095514297485
current: 30
loss: 1.1332333087921143
current: 40
loss: 1.0270366668701172
current: 50
loss: 1.0214550495147705
current: 60
loss: 1.1351794004440308
current: 70
loss: 1.05666983127594
current: 80
loss: 1.0401421785354614
current: 90
loss: 1.133112907409668
current: 100
loss: 1.082810878753662
current: 22
test_loss: 1.062837928533554
correct: 0.4473684210526316
Epoch 2
loss: 1.0528086423873901
current: 0
loss: 0.9869252443313599
current: 10
loss: 1.075459599494934
current: 20
loss: 0.9320133328437805
current: 30
loss: 0.9328996539115906
current: 40
loss: 1.055582046508789
current: 50
loss: 1.0753037929534912
current: 60
loss: 1.0499595403671265
current: 70
loss: 1.0384548902511597
current: 80
loss: 1.0355240106582642
current: 90
loss: 1.069126844406128
current: 100
loss: 1.0905238389968872
current: 22
test_loss: 1.0019849091768265
correct: 0.65789